In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

    # read prices and dates
with open('stock_data/stocks_prices.json', 'r') as file:
    dict_stocks = json.load(file)

ibovespa = dict_stocks['BOVA11']
ibovespa_2 = []
for chave, valor in ibovespa.items():
    ibovespa_2.append(valor)

ibovespa = np.array(ibovespa_2)

# indicadores
indicador1 = []
indicador2 = []

for i in range(20, len(ibovespa)):
    indicador1.append((ibovespa[i] / ibovespa[i-20:i+1].max() - 1) * 100)
    indicador2.append((ibovespa[i] / ibovespa[i-20:i+1].min() - 1) * 100)

indicador1 = np.array(indicador1)
indicador2 = np.array(indicador2)

print(ibovespa[20:].shape)
print(indicador1.shape)
print(indicador2.shape)

df = pd.DataFrame({'Preco': ibovespa[20:], 'Fundo': indicador1, 'Topo': indicador2})
df.to_excel('/Users/gabrielassuncao/Desktop/estudo.xlsx', index=False)

(1496,)
(1496,)
(1496,)


In [2]:
from sklearn.cluster import KMeans

def calculate_inertia(column, k_range):
    inertias = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(column)
        inertias.append(kmeans.inertia_)
    return inertias

k_range = range(1, 11)
inertias_fundo = calculate_inertia(df[['Fundo']], k_range)
inertias_topo = calculate_inertia(df[['Topo']], k_range)

print(df.shape)
fundo = np.array(df['Fundo'])
fundo = fundo.reshape((1496, 1))
topo = np.array(df['Topo'])
topo = topo.reshape((1496, 1))

k = 8
kmeans = KMeans(n_clusters=k, random_state=42)
df['ClusterF'] = kmeans.fit_predict(fundo)
df['ClusterT'] = kmeans.fit_predict(topo)

# Exibir os primeiros resultados para verificar a distribuição dos clusters
df.head()

(1496, 3)


,Preco,Fundo,Topo,ClusterF,ClusterT
0,82.00,-0.786449,9.187750,2,7
1,82.55,-0.120992,9.628154,2,7
2,81.24,-1.705989,7.460317,7,3
3,79.20,-4.174229,4.761905,4,0
4,81.15,-1.814882,7.341270,7,3


In [3]:
retorno_dias = 40
df['Retorno_20d'] = (df['Preco'].shift(-retorno_dias) / df['Preco'] - 1) * 100
df = df.iloc[:-retorno_dias, :]

estatisticas_clusterf = df.groupby('ClusterF')['Retorno_20d'].describe()
estatisticas_clustert = df.groupby('ClusterT')['Retorno_20d'].describe()
estatisticas_clusterf_fundo = df.groupby('ClusterF')['Fundo'].describe()
estatisticas_clustert_topo = df.groupby('ClusterT')['Topo'].describe()

new_df = estatisticas_clusterf[['count', 'mean', '50%']]
new_df = new_df.assign(mean_fundo=estatisticas_clusterf_fundo['mean'])
new_df_sorted = new_df.sort_values('50%', ascending=False)
new_df_sorted

,count,mean,50%,mean_fundo
ClusterF,,,,
5,8.0,16.686792,19.033440,-40.304698
1,11.0,13.380946,16.228984,-29.839126
3,42.0,5.012063,8.259491,-13.360003
0,203.0,3.395125,2.665643,-5.449185
2,492.0,1.556293,2.551890,-0.273785
6,114.0,2.473215,2.349870,-8.189464
4,279.0,0.263540,1.067961,-3.580215
7,307.0,0.761429,0.507259,-1.840740


In [4]:
new_df = estatisticas_clustert[['count', 'mean', '50%']]
new_df = new_df.assign(mean_topo=estatisticas_clustert_topo['mean'])
new_df_sorted = new_df.sort_values('50%', ascending=False)
new_df_sorted

,count,mean,50%,mean_fundo
ClusterT,,,,
4,20.0,15.309560,14.747758,22.779916
5,39.0,9.947169,6.960557,15.651689
1,88.0,4.496987,3.769539,11.698823
3,221.0,1.958492,2.391284,6.881350
0,256.0,1.300076,2.366898,4.638882
2,381.0,1.550688,1.185731,0.461276
6,287.0,0.022051,0.527357,2.532471
7,164.0,0.504133,-0.098219,8.998917
